<a href="https://colab.research.google.com/github/GDS-Education-Community-of-Practice/DSECOP/blob/daleas_module/Connecting_MonteCarlo_to_ModernAI/N1_IntroToMonteCarlo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Notebook 1: Introduction to Monte Carlo Methods and Markov Chains
Ashley Dale


---


In this notebook, you will learn the following concepts:

*   The difference between a model and a simulation
*   The definition of a Markov Chain, and how to implement one
*   How to implement Monte Carlo integration
*   How to evaluate simulation results using standardized metrics

These concepts are useful review for the next notebook on the Ising Model.


#### Setup Python Environment


In [3]:
import numpy as np # used for mathematical operations
import matplotlib.pyplot as plt # used to visualize results
from tqdm import trange # prints a nice loading bar for the notebook

#Background Information

The main computational approach presented in this module is known as ***The Metropolis-Hastings Algorithm***, and was [presented in a paper](https://aip.scitation.org/doi/abs/10.1063/1.1699114) authored by Nicholas Metropolis, Ariana Rosenbluth, Marshall Rosenbluth, Augusta Teller, and Edward Teller.  The authors [developed the algorithm](https://aip.scitation.org/doi/abs/10.1063/1.1632112) during their time at Los Alamos National Laboratory in New Mexico USA, and it was specifically designed to run on [MANIAC, an early vacuum tube computer](https://discover.lanl.gov/news/0412-maniac/).  MANIAC was slow compared to modern machines; in a game of chess, [it required 20 minutes between moves](https://discover.lanl.gov/news/0412-maniac/).  However, this computational approach allowed a breakthrough in physics: a way to predict a physical outcome without having an analytical solution to the differential equation that describes a system's state.

The following sections will walk you through basic ideas and definitions required for understanding the computational approach.  For more on the physics, continue to **Notebook 2: Ising Model Simulation**. 



##Modeling vs. Simulation

In general, a *model* is an object that represents a system; it is not the system itself.  Examples include mathematical equations, mechanical objects such as dice or an [orrery](https://en.wikipedia.org/wiki/Orrery), and deep-neural networks (DNN) used in machine learning.

A *simulation* takes a model and makes predictions about the system by varying model parameters.  See the table below for some examples on how models and simulations relate.

|Original System | Model | Parameter varied during simulation | Simulation Prediction |
|--- | --- | --- | --- |
|Object in free fall| $\Delta y = v_o \Delta t + 0.5 a(\Delta t)^2$ <br> where $\Delta y$ is the change in position, <br> $a$ is the acceleration, <br> $v_o$ is the initial velocity and <br > $t$ is the change in time. | $\Delta t$, change in time | Change in position $\Delta y$ after time $\Delta t$
|Customer Ice Cream Selection | Dice | Dice physical configuration | Number on dice correlates to ice cream flavor |
|Solar System| Orrery | Planet objects start in an initial position, <br> a handle is cranked to change this position | New planetary positions |
|Human identifies cat or dog | DNN | The input image to the network is changed | The input is labeled "cat" or "dog" |



Validating a model and its simulation can be extremely challenging.  Different approaches depend on the data and information available to you.  Consider the following situations:

**Case 1**: You have simulation results and experimental results. This is an ideal case for a physicist, as physics models are designed to predict physical phenomena.  

**Case 2**: You have simulation results for one simulation and no experimental results.  In this case, you will want to see how self-consistent your simulation results are, <br> and if the outputs fall within an expected range.  <br>

**Case 3**: You have simulation results for multiple kinds of simulations, but no experimental results.  Here, you will want to compare whether or not the simulations predict the same thing.

The metric used to evaluate a simulation will depend on which case you have.  See the table for different metrics:


| Metric | Equations | Notes |
| ----- | ----- | ----- |
| Mean<br>Square<br>Error (MSE) | $\frac{1}{n}\sum_{i=1}^n \left(Y_i - \hat{Y}_i\right)^2$ where<br> $Y_i$ is the expected value and $\hat{Y}_i$ is the predicted value| The average distance <br> between two datasets |
| Percent<br>Error |$\left (Y_i - \hat{Y}_i \right)/Y_i \times 100 $ where<br> $Y_i$ is the expected value and $\hat{Y}_i$ is the predicted value |Can only be used when <br>there is an accepted value, i.e. Case 1|
| Percent<br>Difference | $ 2\left| Y_i - \hat{Y_i} \right| / (Y_i + \hat{Y_i})$ where<br> $Y_i$ is the first predicted value and $\hat{Y}_i$ is the second predicted value| Good for Cases 2 and 3 |
| Standard<br>Error | $\frac{\sigma}{\sqrt{n}}$ where $\sigma$ is the standard deviation and <br>$n$ is the number of data points considered in the standard deviation | Good for Case 3 |



## Stochastic Systems & Probabilistic Model Review
Many real world systems do not have a single outcome for a single input, and there is some uncertainty as to which outcome you will have.  These systems are called *stochastic*.  They can be modeled using probability distributions.

---
>**An Example of a Stochastic System and Probability Model (Featuring my Cat)** <br>
>
>Consider my cat. When I greet him in the morning, he usually wants to be pet immediately, but sometimes he wants to be fed breakfast first.  After collecting much data about my cat, I can model this behavior using an equation that has no theory or assumptions behind it (it is *empirical*).  The probability that my cat wants to be pet first is:
>
> <center>$P(Cat\ wants\ pets) = \frac{\#\ Times\ He\ Asks\ For\ Pets}{\#\ Times\ I\ Say\ Good\ morning}$</center>
>
>Because he only wants pets or breakfast, I can also write this equation:
>
><center>$P(Cat\ wants\ breakfast) = 1 - P(Cat\ wants\ pets)$</center>
>
>This means there is a 100% chance that he wants either pets or breakfast, so 
>
><center>$P(Cat\ wants\ breakfast\ OR\ pets) = P(Cat\ wants\ breakfast) + P(Cat\ wants\ pets) = 1$</center>
>
>For the same input when I see him in the morning, my cat has two outputs.  (That's it.  He is not a very smart cat.)  However, after living with him for 18 years, we are pretty good friends, and I know that most mornings he prefers to be pet before eating his food.  In fact, I would say that this is the case 75% of the time:
>
><center>$P(Cat\ wants\ pets) = 0.75$</center>
>
>And this means that I automatically go to pet him first before I head to the kitchen to make him breakfast.  He's done a good job conditioning me.

---

In the example above, the model is very simple since it only has two outcomes, so I would use a [Bernoulli distribution](https://en.wikipedia.org/wiki/Bernoulli_distribution) to model my cat.  Here is a reminder of some other common probability distributions.  If you are not familiar with these, take a few minutes to review them:


Distribution Name | Distribution Equation | Comments
--- | --- | ---
[Normal](https://en.wikipedia.org/wiki/Normal_distribution)|$f(x) = \frac{1}{\sigma \sqrt{2 \pi}}e^{-\frac{1}{2} \left(\frac{x-\mu}{\sigma}\right)^2} $|Gaussian!  $\mu$ is the mean and $\sigma$ is the standard deviation
[Uniform](https://en.wikipedia.org/wiki/Continuous_uniform_distribution)|$f(x) = \frac{1}{b-a}$ for $a \le x \ge b$ and <br> 0 everywhere else |All outcomes in the range of $a$ to $b$ are equally likely
[Boltzmann](https://en.wikipedia.org/wiki/Boltzmann_distribution)|$p_i = \frac{e^{-E_i/(kT)}}{\Sigma^M_{j=1} e^{E_j/(kT)}}$|The probability the system is in a specific state depends on the temperature



## Markov Chains

A *Markov Chain* is a stochastic model that describes a sequence.  The probability of the next item in the sequence is only determined by the last item in the sequence.
[<img src=https://upload.wikimedia.org/wikipedia/commons/2/2b/Markovkate_01.svg alt="Picture of two-state Markov Chain with probability of transitioning between states.  Source: Wikipedia" align="left" width=300>](https://commons.wikimedia.org/wiki/File:Markovkate_01.svg)

In the figure to the left (*source: [Wikipedia](https://commons.wikimedia.org/wiki/File:Markovkate_01.svg)*), the Markov Chain sequence has two items: state $E$ and state $A$.  The arrows show how to get from one state to the next, and the probability of making each state change is next to each arrow.  Here, we can say that there is a higher probability of staying in state $A$ than in state $E$ because the probability value next to the arrow from $E$ to $A$ is 0.7 (compared to the probability value of 0.3 from $E$ to $E$), and the probability  value from $A$ to $A$ is 0.6 (compared to the probability value of 0.4 for $A$ to $E$).  It is always more likely to be in state $A$ than in state $E$, even if we start the system in state $E$.  

The most important thing to notice is that the Markov Chain does not keep track of how long it has been in a particular state, or what states it has already been to.  There is no memory of where the system has already been; the only thing you need to make a prediction for what state comes next in the sequence is knowledge of the current system state.  This makes it a very powerful modeling approach; the key is correctly generating the probabilities next to the arrows for your particular use case.



##Monte Carlo Sampling

*Monte Carlo Sampling* is useful when you have a model for your probability distribution like the ones in the table above, but do not know the exact probability of the outcome you are trying to predict.  Basically, you guess the probability numbers next to the orange arrows in the Markov Chain, and see if your guess makes your system model behave like the real-world one.

---
>**An Example of Monte Carlo Sampling (Also Featuring My Cat)**
>
>This is where I admit that I have not, in fact, recorded my cat's preferences every morning for the past 18 years and then compiled statistics for my Bernoulli distribution above.  However, I don't actually need to in order to have an experimentally valid prediction for his behavior.  My experimental procedure might go like this:
>
> 1. Flip a coin before I do anything else
> 2. If the coin lands "heads" up, I will write down his response in my lab notebook
> 3. If the coin lands "tails" up, I won't bother to write anything down
>
>My decision to collect data is randomly decided by the coin toss.  If my coin has 1:1 odds of landing heads or tails, this means I am collecting data only 50% of the time.  The technical term for this is *randomly sampling the distribution*, because I am not collecting every data point – only some of them – and I am choosing which ones to keep randomly.  
>
> 4. I decide how many times to repeat steps 1-3.  
>
>The more data I collect, the better my approximation of the actual probability for my cat's behavior.  However, there are practical considerations.  I can't collect data forever; but too few data points will not let me make good predictions.  The exact way I determine how many points to collect might depend on how confident I need to be about my prediction, how quickly my data forms a nice pattern, etc.
---

The combination of Markov Chains with Monte Carlo sampling is extremely powerful, and is what we will explore in the next notebook.


#Exercises

Complete the exercises below.

## Programming Exercise 1: Markov Chain

Below is a partially completed program that simulates the Markov Chain model shown in the figure above.  

a) Complete the program, and report the average number of steps it takes to change to state $A$ when starting in state $E$.  Include the uncertainty (i.e. [standard error](https://en.wikipedia.org/wiki/Standard_error)) in your reported number.  You should make additional code cells in your notebook to keep track of your data and present your results.



---
Here is the transition probability table:

State Change | Probability 
--- | ---
EE | 0.3
EA | 0.7
AE | 0.4
AA | 0.6

In [2]:
# Here are some variables to get you started

# Decide how many steps to make the simulation run
num_steps = 100

# Define a starting state: 
current_state = 'E' 

# Define an ending state:
end_state = 'A'

In [1]:
# Decide on a probability distribution function for the simulation 
# (uncomment one of the probability lines in the function): 

def get_probability(): 
  # This function returns a probability value when called

  # --> This is a sample from a uniform distribution from numpy
  
  #prob = np.random.uniform(0, 1, 1)

  # --> This is a sample from a normal distribution from numpy
  # --> It is centered at 0.5 to match the default uniform distribution
  
  #prob = np.random.normal(loc=0.5, size=1)
  
  return prob

In [ ]:
# Now we run the simulation

for step in range(num_steps):
  p = get_probability()

  # --- HERE IS THE STATE MACHINE CODE --- #

  # --> The first logical case is written for you

  # If the state is currently 'E' 
  # and transition probability is less than or equal to 0.3
  if current_state == 'E' and p<=0.3:
    
    # stay in state 'E'
    current_state = 'E'
  
  # --> Now you write the next cases:

  # if the state is currently 'E' 
  # and the transition probability is greater than 0.3
  if current_state == and p> :
    
    # move to state 'A'
    current_state = 

  # if the state is currently 'A'
  # and the transition probability is less than equal to 0.4
  if current_state == and p<= :
    
    # move to state 'E'
    current_state =

  # if the state is currently 'A'
  # and the transition probability is greater than 0.4
  if current_state == and p> :

    # stay in state 'A'
    current_state = 
  # --- STATE MACHINE CODE ENDS --- #

  # Stop updating when the system has transitioned to the end state
  if current_state == end_state:
    break

print("Ending state achieved after "+str(step+1)+" iterations")


In [ ]:
# Store your results here:

data_EtoA = []#STORE YOUR VALUES IN THIS LIST
mean_EtoA = #<WRITE YOUR AVERAGE VALUE CALCULATION HERE>
stderr_EtoA = #<WRITE YOUR STANDARD ERROR CALCULATION HERE>

print(f"# of tries to transition from E to A: {mean_EtoA} +/- {stderr_EtoA}")

data_AtoE = []#STORE YOUR VALUES IN THIS LIST
mean_AtoE = #<WRITE YOUR AVERAGE VALUE CALCULATION HERE>
stderr_AtoE = #<WRITE YOUR STANDARD ERROR CALCULATION HERE>

print(f"# of tries to transition from A to E: {mean_AtoE} +/- {stderr_AtoE}")

b) The program above can be written much more simply, and the experiment can be done more efficiently by defining a function for the whole experiment that lets the user automate the results. 


*   Define a new function "MarkovChainExperiment" (outlined for you below) that takes as an inputs the number of times to run the experiment, the initial state, and the final state and returns a list containing how many steps it took to change the state for each time the experiment was run.  The function should let the user specify how many times to simulate the Markov Chain, run the simulation that number of times, and then return all of the results at once.  *Hint*: You will need to add another for loop.

*   Rewrite the state machine code using only two if-statements

*   Test your function by calling it in a new cell, and making a histogram of the results for 1 million experiments each of "E to A" and "A to E".


In [ ]:

def MarkovChainExperiment():

  # Begin by initializing the 
  # current state with the value passed into the function
  current_state = 

  # define a variable to track results
  results = [] #this is a python list data type to make things easy

  # --> Now run the experiment repeatedly:

    # for each experiment trial:

        # for each step in the range of steps to try:
        for n in range(num_steps):
          p = get_probability()

          # --- HERE IS THE STATE MACHINE CODE --- #







          # --- STATE MACHINE CODE ENDS --- #

          # Stop updating when the system has transitioned to the end state
          if current_state == end_state:
            # update the results list by appending
            # the number of steps just finished to the end of the list

            break

            
  return results


In [ ]:
# Call the function here with the necessary parameters

data_EtoA = MarkovChainExperiment()
data_AtoE = MarkovChainExperiment()

Create a figure showing the histogram distribution of the number of steps it took to change states for both `data_EtoA` and `data_AtoE`.  

* [How to make histograms with Matplotlib.pyplot](https://pythonspot.com/matplotlib-histogram/)

In [4]:
# Put your plotting code here

##Programming Exercise 2: Monte Carlo Integration

Let's dive a little bit deeper into the idea of Monte Carlo sampling through an exercise where Monte Carlo sampling is used to improve a numerical integration result.

Numerical integration uses the rectangle approximation to find the area under a curve.  The analytical notation we are used to seeing for a definite integral

<center> $F = \int_a^b f(x) dx $ </center>

can be expressed as a [numerical approximation that adds up $n$ rectangles under the curve $f(x)$](https://mathworld.wolfram.com/NumericalIntegration.html).  The more rectangles used to calculate the area, the better the approximation becomes.

Monte Carlo Integration improves this approach by randomly picking which rectangles to add up next and approximating $F$ as $\langle F^N \rangle$:

<center> $\langle F^N \rangle = \frac{b-a}{N-1} \sum_{i=0}^{N} f(X_i)$ </center>

where $N$ is the number of times a new value $X_i$ is chosen from a probability distribution for range $a$ to $b$.  Therefore, 

<center> $lim_{N→∞} \langle F^N \rangle = F $ </center>

The question becomes, what is the best way to choose $X_i$ to match the real system?  The goal is to get the best approximation as quickly as possible.

---
###For this exercise, do the following:

1. Choose a definite expression from the table below (or your instructor might tell you which one to pick):

>||Function|Interval|Notes|
|---|:-----------------------:|---|---|
|1. |$x = \int_0^1 -10t\ dt$| $t$ is time; $t = [0, 1]$ |Gives position at time $t$ for this system|
|2. |$\Delta S = \int_{300}^{400}\frac{mc}{T}\ dT $|$m$ is mass; $m=1$ kg<br> $c$ is specific heat capacity; $c = 4190$ J/kg K<br>$T$ is temperature; $T = [300, 400]$|Change in entropy for thermal processes|
|3. |$\Phi = \int_1^2 \frac{Q}{4 \pi \epsilon_o r^2} dr$|$r$ is distance; $r = [1, 2]$<br>$\epsilon_o$ is the Permittivity of Free Space<br>$Q$ is the charge; $Q = 1$C|$\Phi$ is the electric potential energy <br>gained by moving along line $r$|
|4. |$I = \int_0^\infty \frac{2 \pi h c^2}{\lambda^5(e^{hc/\lambda k T} - 1)}\ d\lambda$|$h$ is Planck's constant<br> $c$ is speed of light <br> $k$ is Boltzmann's Constant <br> $T$ is the absolute temperature; T = 400K <br> $\lambda$ is wavelength; $\lambda = [0, \infty]$|Planck's radiation law; <br>Integrating gives Stefan Boltzmann Law|

2. Analytically integrate it for the region and values provided, and record your answer in the `analytical_result` variable below




In [ ]:
analytical_result = 

3. Write the expression to be integrated as a python function.  For example, if I want to integrate the expression

<center> $F = \int 3x^2 + 17\ dx$</center>

then my integrand is

<center> $f(x) = 3x^2 + 17$</center>

and I would write the following function:
```
def integrand(x):
    f_x = 3*np.power(x, 2) + 17
    return f_x
```
This function takes `x` as my function argument, and returns the calculated value `f_x`.  Note that I am not yet evaluating the limits of my integrand.

In [5]:
# Helpful constants:
pi = np.pi #unitless
c = 2.99E8 #m/s
h = 6.62607015E-34 #J
k = 1.380649E-23 #J/K
epsilon = 8.854187817E-12 #F/m
sigma = 5.6704E-8 #W/(m^2 K^4)

In [ ]:
def integrand(x):
  f_x = #<DEFINE YOUR INTEGRAND HERE>
  return f_x

4. Randomly choose values for `x` from a distribution between the limits of the definite integral. 
>*Hint*: if one of your limits is $\infty$, it is okay to approximate it with a large number.  Another way to do it is to plot [x, f(x)] and visually estimate the most important region of your integration.

In [ ]:
lower_limit = 
upper_limit = 
num_x_vals = 
x_vals = np.random.uniform(lower_limit, upper_limit, num_x_vals)

5. Calculate the `f_x` values:

In [ ]:
y = integrand(x_vals)

6. Add them all together using the approximation: 
<center> $\langle F^N \rangle = \frac{b-a}{N-1} \sum_{i=0}^{N} f(X_i)$ </center>


In [ ]:
# Write code to calculate the equation above here
approx_int = #<WRITE YOUR EQUATION HERE>
print(f"The Monte Carlo approximation is {approx_int}")

7. Calculate the error between the `approx_int` and the `analytical_result` variables using one or more of the metrics discussed above

In [ ]:
error = 
print(f"The error is {error}")

8. Finally, we want to visualize how the error decreases as the number of random trials `num_x_vals` increases.  Write code to the do the following:

* Using the error metric you decided on above, write a for-loop that calculates the error as a function of the number of points you sampled.  For example, calculate the error when you summed two values of $\langle F^N \rangle$, then calculate the error for three summed values of $\langle F^N \rangle$, and so on until you have calculated the errors for the full range of $\langle F^N \rangle$.

* IMPORTANT: You do not need to re-do the experiment to calculate this analysis.  Use the `y` list variable.

* Make a figure showing how the error changes with the number of values in the sum.

* Answer these questions: What can you tell about the trend in the error?  Would your approximation benefit from additional calculations (increasing the number of `num_x_vals`?  Could you have done fewer calculations?

In [ ]:
# Complete the code in this cell:

# Make a variable to store your error calculations in
error_data = []

# First I loop over all of the values of N
for idx in trange(2, num_x_vals):#I am starting my index at 2 to avoid division by zero

  #Next, adjust the integral expression so it doesn't use all of the points at once
  approx_int = 

  # Calculate the error
  err = 

  # Then save the error value to the error_data array
  error_data.append(perre)

In [ ]:
# Add your plotting code here

*Answer the questions here:*

##Question 1: Model vs Simulation
In your own words, describe the difference between a model and a simulation.  Give your own example of a model, and how you would simulate it.

*Enter your answer here*

##Question 2: Markov Chain
In your own words, describe a Markov Chain and its properties. Give your own example of a stochastic system and how you would implement it.

*Enter your answer here*

# Additional Readings
1. [Metropolis, Nicholas, et al. "Equation of state calculations by fast computing machines." The journal of chemical physics 21.6 (1953): 1087-1092.](https://aip.scitation.org/doi/abs/10.1063/1.1699114)

2. [Van Ravenzwaaij, Don, Pete Cassey, and Scott D. Brown. "A simple introduction to Markov Chain Monte–Carlo sampling." Psychonomic bulletin & review 25.1 (2018): 143-154.](https://link.springer.com/article/10.3758/s13423-016-1015-8?wt_mc=Other.Other.8.CON1172.PSBR%20VSI%20Art09%20&%20utm_medium=other%20&%20utm_source=other%20&%20utm_content=2062018%20&%20utm_campaign=8_ago1936_psbr%20vsi%20art09)

3. [Victor Cumer. "The basics of Monte Carlo integration". Towards Data Science (2020)](https://towardsdatascience.com/the-basics-of-monte-carlo-integration-5fe16b40482d)

4. [Nicholas Lewis. "70 years of electronic computing." Los Alamos National Laboratory (2022)](https://discover.lanl.gov/news/0412-maniac/)

5. [Rosenbluth, Marshall N. "Genesis of the Monte Carlo algorithm for statistical mechanics." AIP Conference Proceedings. Vol. 690. No. 1. American Institute of Physics, 2003.](https://aip.scitation.org/doi/abs/10.1063/1.1632112)